# The Chinese Remainder Theorem

**Module 04** | 04-number-theory-rsa

*CRT_list(), solving congruences*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **the chinese remainder theorem**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Completion of [Euler's Totient and Fermat's Theorem](04c-euler-totient-fermats-theorem.ipynb).
- Concepts and notation introduced in the previous notebook.

## Simultaneous Congruences

In [ ]:
# Solve: x = 2 mod 3, x = 3 mod 5, x = 2 mod 7
x = CRT_list([2, 3, 2], [3, 5, 7])
print(f'x = {x}')
print(f'Check: {x%3}, {x%5}, {x%7}')

## CRT and Isomorphism

In [ ]:
# Z/15Z is isomorphic to Z/3Z x Z/5Z when gcd(3,5)=1
for x in range(15):
    print(f'{x} -> ({x%3}, {x%5})')

## CRT in RSA

In [ ]:
# CRT speeds up RSA decryption by splitting mod n = p*q
# TODO: demonstrate CRT-based RSA decryption

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **the chinese remainder theorem**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*

**Next:** [RSA Key Generation](04e-rsa-key-generation.ipynb)